In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#variaveis de gating soma

def gating_soma(Vs):
    alpha_h = 128*np.exp(-(Vs+0.043) / 0.018)
    beta_h = 4e3/(1+np.exp(-200*(Vs+0.020)))
    beta_n = 250*np.exp(-25*(Vs+0.040))

    if Vs == -0.0469:
        alpha_m = 320/0.25
    else:
        alpha_m = ( 320*1e3*(Vs+0.0469) ) / (1-np.exp(-250*(Vs+0.0469)))

    if Vs == -0.0249:
        alpha_n = 16/0.2
    else:
        alpha_n = ( 16e3*(Vs+0.0249) ) / (1-np.exp(-200*(Vs+0.0249)))

    if Vs == -19.9:
        beta_m = 280/0.2
    else:
        beta_m = (280e3*(Vs+0.0199))/(np.exp(200*(Vs+0.0199))-1)

    return alpha_m, alpha_h, alpha_n, beta_m, beta_h, beta_n

In [4]:
#variaveis de gating dendrito

def gating_dend(Vd, Ca):
    alpha_mca = 1600/(1+np.exp(-72*(Vd-0.005)))
    alpha_mkahp = min(20,20e3*Ca)
    beta_mkahp = 4

    if Vd == -0.0089:
        beta_mca = 20/0.2
    else:
        beta_mca = 20e3*(Vd+0.0089)/(np.exp(200*(Vd+0.0089))-1)

    if Vd > -0.010:
        alpha_mkca = 2e3*np.exp(-(0.0535+Vd)/0.027)
        beta_mkca = 0
    else:
        alpha_mkca = np.exp( (Vd+0.050) / 0.011 - (Vd+0.0535) / 0.027 ) / 0.018975
        beta_mkca = (2e3*np.exp(-(0.0535+Vd)/0.027) - alpha_mkca)

    return alpha_mca, alpha_mkca, alpha_mkahp, beta_mca, beta_mkca, beta_mkahp

In [ ]:
dt = 10e-6
tmax=2

E_L = -0.060   # leak reversal potential
E_Na = 0.060   # reversal for sodium channels
E_K = -0.075   # reversal for potassium channels
E_Ca = 0.080   # reversal for calcium channels

S_frac = 1/3        # fraction of total membrane area that is soma
D_frac = 1-S_frac   # rest of area is dendritic

# Conductance values for somatic channels follow
G_LS = 5e-9*S_frac     # somatic leak conductance in Siemens
G_Na = 3e-6*S_frac     # sodium conductance (Soma)
G_K = 2e-6*S_frac      # potassium conductance (Soma)

# Conductance values for dendritic channels follow
G_LD = 5e-9*D_frac         # dendritic leak conductance in Siemens
G_Ca = 2e-6*D_frac         # calcium conductance (Dendrite)
G_KAHP = 0.04e-6*D_frac    # Potassium conductance to generate after-hyperpolarization
G_KCa = 2.5e-6*D_frac      # calcium-dependent Potassium conductance

G_Link = 20e-9 # conductance linking dendrite and soma

tau_Ca = 50e-3              # time constant for buffering of calcium
convert_Ca = 0.25e7/D_frac  # conversion changing calcium charge entry per unit area into concentration

CmS = 100e-12*S_frac     # somatic membrane capacitance in Farads
CmD = 100e-12*D_frac     # dendritic membrane capacitance in Farads

In [ ]:
t = np.arange(0, tmax, dt)           # time vector
VS=np.zeros(np.size(t))     # somatic voltage vector
VD=np.zeros(np.size(t))     # dendritic voltage vector
VS[1] = E_L           # set the inititial value of somatic voltage
VD[1] = E_L           # set the inititial value of dendritic voltage


Ca = np.zeros(np.size(t))  # dendritic calcium level (extra Ca above base level)
Ca[1] = 0          # initialize with no (extra) Ca in cell.

I_LD = np.zeros(np.size(t))        # leak current in dendrite
I_LS = np.zeros(np.size(t))        # leak current in soma
I_Na = np.zeros(np.size(t))       # sodium current (soma)
I_K = zeros(size(t))        # potassium current (soma)
I_Ca = zeros(size(t))       # calcium current (dendrite)
I_KAHP = zeros(size(t))     # after-hyperpolarization current (dendrite)
I_KCa = zeros(size(t))      # calcium-dependent potassium current (dendrite)

n=zeros(size(t))   # n: potassium activation gating variable
m=zeros(size(t))   # m: sodium activation gating variable
h=zeros(size(t))   # h: sodim inactivation gating variplot(t,V)able
n(1) = 0.4         # initialize near steady state at resting potential
h(1) = 0.5         # initialize near steady state at resting potential

mca=zeros(size(t))     # Ca current activation gating variable
mkca=zeros(size(t))    # K_Ca current activation gating variable
mkahp = zeros(size(t)) # K_AHP current activation gating variable
mkahp(1) = 0.2        # initialize near steady state at resting potential
mkca(1) = 0.2          # initialize near steady state at resting potential
Ca(1) = 1e-6           # initialize near steady state at resting potential

Itot=zeros(size(t))     # in case we want to plot and look at the total current